# [ Pré-processamento ] Heurísticas computacionais para extração deconhecimento em problemas de maratona deprogramação

Antes de elaborar o modelo com os dados obtidos do URI, é necessário realizar o preprocessamento, a fim de que os dados fornecidos como entrada para os algoritmos de aprendizado (estabelecidos durante a concepção do modelo) estejam não estejam corrompidos ou com discrepâncias que possam vir a prejudicar o modelo.

### Importando as bibliotecas necessárias

Antes de começar, é necessário importar algumas bibliotecas para tratar os dados.

In [1]:
import os
import json
import pandas as pd

### Declarando constantes de trabalho

Criação de "variáveis de ambiente" para auxiliar a obtenção de dados.

In [2]:
# Langs? {PT, EN, ES}
LANG_SLUG = "PT"
ORIGINAL_DATASET_DIR = "../URI_data/URIScraper_%s/" % (LANG_SLUG)
PREPROCESSED_DATA_DIR = "../Datasets/"

# JSON files
JSON_DATASET_FILES = []

### Obtendo os arquivos JSON

Obtenção dos arquivos JSON contidos no diretório do dataset atual.

In [3]:
for file in os.listdir(ORIGINAL_DATASET_DIR):
    if file.endswith(".json"):
        JSON_DATASET_FILES.append(os.path.join(ORIGINAL_DATASET_DIR, file))
        
#Prints 10th first elements in the list
print(JSON_DATASET_FILES[:10])

['../URI_data/URIScraper_PT/1001.json', '../URI_data/URIScraper_PT/1002.json', '../URI_data/URIScraper_PT/1003.json', '../URI_data/URIScraper_PT/1004.json', '../URI_data/URIScraper_PT/1005.json', '../URI_data/URIScraper_PT/1006.json', '../URI_data/URIScraper_PT/1007.json', '../URI_data/URIScraper_PT/1008.json', '../URI_data/URIScraper_PT/1009.json', '../URI_data/URIScraper_PT/1010.json']


### Criando o DataFrame

Criação do dataframe contendo todos os problemas obtidos.

In [4]:
problems = pd.DataFrame(columns=['name', 'category', 'level', 'description', 'input', 'output', 'has_images', 's_level', 'submissions', 'solved', 'ratio'])

for file in JSON_DATASET_FILES:
    #problems.append(pd.read_json(file), ignore_index=True)
    with open(file, 'r', encoding="utf8") as f:
        json_data = json.loads(f.read())
        json_df = pd.DataFrame(data={'name' : json_data['name'], 'category' : json_data['category'], 'level' : json_data['level'], 'description' : json_data['description'], 'input' : json_data['input'], 'output' : json_data['output'], 'has_images' : json_data['has_images'], 's_level' : json_data['statistics']['level'], 'submissions' : json_data['statistics']['submissions'], 'solved' : json_data['statistics']['solved'], 'ratio' : json_data['statistics']['ratio']},
                              index=[json_data['id']])
        problems = problems.append(json_df)

Verificando o novo dataframe criado

In [5]:
print("Head:")
print(problems.head(10)[['name', 'category', 'level']])
print("\nTail:")
print(problems.tail(10)[['name', 'category', 'level']])

Head:
                     name   category level
1001  Extremamente Básico  Iniciante     5
1002      Área do Círculo  Iniciante     5
1003         Soma Simples  Iniciante     2
1004      Produto Simples  Iniciante     1
1005              Média 1  Iniciante     4
1006              Média 2  Iniciante     2
1007            Diferença  Iniciante     2
1008              Salário  Iniciante     4
1009    Salário com Bônus  Iniciante     3
1010      Cálculo Simples  Iniciante     3

Tail:
                          name                  category level
2952        A Vida Sustentável                 Iniciante     1
2953          Mais um Joguinho                Matemática     1
2954                    O Jogo                   Strings     1
2955                Truuuuuco!                Matemática     1
2956  Derivada de 13 Variáveis                Matemática     1
2957                    Sir Vô                    Ad-Hoc     1
2958          O Rolê Bad Vibes  Estruturas e Bibliotecas     1
2959      

### Exportação de dados sem tratamento primário

In [6]:
rawDataPath = "%s%s_00_raw_data.csv" % (PREPROCESSED_DATA_DIR, LANG_SLUG)
problems.to_csv(rawDataPath, sep=";")
print("DataFrame shape:")
print(problems.shape)
del rawDataPath

DataFrame shape:
(1944, 11)


### Pré-processamento primário

Remoção de problemas sem descrição

In [7]:
# Removes problems without a proper description
removed_items = problems[problems["description"] == "undefined"].index
problems.drop(removed_items, inplace=True)
print("Removed items:")
print(removed_items)

primaryDataPath = "%s%s_01_primary_data.csv" % (PREPROCESSED_DATA_DIR, LANG_SLUG)
problems.to_csv(primaryDataPath, sep=";")
print("DataFrame shape:")
print(problems.shape)

del removed_items
del primaryDataPath

Removed items:
Index(['2602', '2603', '2604', '2605', '2606', '2607', '2608', '2609', '2610',
       '2611', '2612', '2613', '2614', '2615', '2616', '2617', '2618', '2619',
       '2620', '2621', '2622', '2623', '2624', '2625', '2694', '2737', '2738',
       '2739', '2740', '2741', '2742', '2743', '2744', '2745', '2746', '2903',
       '2904', '2905', '2906', '2907', '2908', '2909', '2910', '2911', '2912',
       '2913', '2914', '2915'],
      dtype='object')
DataFrame shape:
(1896, 11)


### Pré-processamento secundário

Remoção de problemas sem descrição de entrada e saída (já removido os problemas sem descrição)

In [8]:
removed_items = problems[(problems["input"] == "undefined") | (problems["output"] == "undefined")].index
problems.drop(removed_items, inplace=True)
print("Removed items:")
print(removed_items)

secondaryDataPath = "%s%s_02_secondary_data.csv" % (PREPROCESSED_DATA_DIR, LANG_SLUG)
problems.to_csv(secondaryDataPath, sep=";")
print("DataFrame shape:")
print(problems.shape)

del removed_items
del secondaryDataPath

Removed items:
Index([], dtype='object')
DataFrame shape:
(1896, 11)


### Pré-processamento terciário

Remoção de problemas sem estatísticas (já removido os problemas sem descrição ou sem dados de entrada ou saída)

In [9]:
removed_items = problems[problems["s_level"] == "undefined"].index
problems.drop(removed_items, inplace=True)
print("Removed items:")
print(removed_items)

terciaryDataPath = "%s%s_03_terciary_data.csv" % (PREPROCESSED_DATA_DIR, LANG_SLUG)
problems.to_csv(terciaryDataPath, sep=";")
print("DataFrame shape:")
print(problems.shape)

del removed_items
del terciaryDataPath

Removed items:
Index(['1812', '1818', '1950', '1995', '2105', '2113', '2209', '2259', '2266',
       '2279', '2282', '2283', '2352', '2645'],
      dtype='object')
DataFrame shape:
(1882, 11)
